# Modelo oficial

- **Objetivo:** Identificar se a pessoa quer ou não trocar de trabalho baseado em suas características

- **Métricas**: 
    - Acurácia
    - Recall
    - Precision
    - F1-score

- **Modelo de ML**: Light Gradient Boosting Machine

## 0. Setup

In [1]:
import pandas as pd
import numpy as np

## 1. Carregando os dados

In [2]:
dados = pd.read_csv(filepath_or_buffer = '../data/raw/aug_train.csv')

dados.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


## 2. Criar as novas features

### 2.1. Agrupar os valores de company_size em PP, P, M e G

In [3]:
def add_feature_company_size(df):
    
    """
    # PP: Até 50 pessoas
    # P: Vai de 50 até 500
    # M: Vai de 500 até 4999
    # G: Acima de 5000
    """
    
    df1 = df.copy()
    
    df1['company_size_cat'] = np.where(dados['company_size'].isin(['<10', '10/49']), 'PP', 
                                       np.where(dados['company_size'].isin(['50-99', '100-500']), 'P',
                                                np.where(dados['company_size'].isin(['500-999', '1000-4999']), 'M',
                                                         np.where(dados['company_size'].isin(['5000-9999', '10000+']), 'GG', 
                                                                  np.nan))))
    
    return df1

### 2.2. Fazer uma feature que divide a quantidade de horas treinadas por 24 (resultados em quantos dias de treinamento ela participou)

In [4]:
def add_feature_training_hours(df):
    
    df1 = df.copy()
    
    df1['days_training_hours'] = df1['training_hours'] / 24
    
    return df1

### 2.3. Criar uma variável categórica que diz se a pessoa é nova ou não no mercado de trabalho. Ex.: Se a pessoa tem 3 ou menos anos de experiência, ela é nova, senão ela é "velha"

In [5]:
def add_feature_experience(df):
    
    df1 = df.copy()
    
    df1['experience_cat'] = np.where(dados['experience'].isin(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9']), 0, 
                                     np.where(dados['experience'].isin(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']), 1, 
                                              np.nan))
    
    return df1

### 2.4. Agrupar os valores de company_type relacionadas a startup

In [6]:
def add_feature_company_type(df):
    
    df1 = df.copy()
    
    df1['company_type_cat'] = np.where(dados['company_type'].isin(['Funded Startup', 'Early Stage Startup']), 1, 
                                     np.where(dados['company_type'].isin(['Pvt Ltd', 'Other', 'Public Sector', 'NGO']), 0, 
                                              np.nan))
    
    return df1

### 2.5. Criando função para identificar nulos em qualquer variável (se for nulo, 1, 0)

In [7]:
def add_feature_null_column(df, col):
    
    df1 = df.copy()
    
    df1['check_null_' + col] = np.where(df1[col].isna(), 1, 0)
    
    return df1

### 2.6. Criando função para identificar nulos em variáveis qualitativas

In [8]:
def add_feature_null_qualitative(df, col):
    
    df1 = df.copy()
    
    df1[col] = np.where(df1[col].isna(), 'Outras', df1[col])
    
    return df1

### 2.7. Criando função para identificar nulos em variáveis quantitativas

In [9]:
def add_feature_null_quantitative(df, col):
    
    df1 = df.copy()
    
    df1[col] = np.where(df1[col].isna(), 99999, df1[col])
    
    return df1

## 3. Criação do modelo

### 3.0. Setup

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer
from sklearn import set_config 
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, Normalizer, QuantileTransformer, StandardScaler, OneHotEncoder
import category_encoders as ce
import lightgbm as lgb


set_config(display = "diagram")

### 3.1. Divisão da base de treino e teste

In [22]:
X = dados.drop(columns = 'target', axis = 1)

y = dados.target

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 19, stratify = y)

In [24]:
print(f'Quantidade de linhas do X_train: {X_train.shape[0]} \n \
Quantidade de linhas do X_test: {X_test.shape[0]}\n \
Quantidade de linhas do y_train: {y_train.shape[0]}\n \
Quantidade de linhas do y_test: {y_test.shape[0]}\
')

Quantidade de linhas do X_train: 13410 
 Quantidade de linhas do X_test: 5748
 Quantidade de linhas do y_train: 13410
 Quantidade de linhas do y_test: 5748


### 3.2. Definindo os passos do Pipeline de Feature Engineering

In [49]:
encoder1 = ce.BackwardDifferenceEncoder()
encoder2 = ce.BaseNEncoder()
encoder3 = ce.BinaryEncoder()
encoder4 = ce.CatBoostEncoder()
encoder5 = ce.CountEncoder()
encoder6 = ce.GLMMEncoder()
encoder7 = ce.HashingEncoder()
encoder8 = ce.HelmertEncoder()
encoder9 = ce.JamesSteinEncoder()
encoder10 = ce.LeaveOneOutEncoder()
encoder11 = ce.MEstimateEncoder()
encoder12 = OneHotEncoder(handle_unknown = "ignore")
encoder13 = ce.OrdinalEncoder()
encoder14 = ce.SumEncoder()
encoder15 = ce.PolynomialEncoder()
encoder16 = ce.TargetEncoder()
encoder17 = ce.WOEEncoder()
encoder18 = ce.QuantileEncoder()
encoder19 = MaxAbsScaler()
encoder20 = MinMaxScaler()
encoder21 = Normalizer()
encoder22 = QuantileTransformer()
encoder23 = StandardScaler()

model = lgb.LGBMClassifier(random_state = 42)

/home/rafael/Documentos/Github/hr_analysis/env/lib/python3.10/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/rafael/Documentos/Github/hr_analysis/env/lib/python3.10/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [50]:
features_qual = list(dados.select_dtypes(include = ['object']).columns)
features_quant = list(dados.drop(columns = ['enrollee_id', 'target'], axis = 1).select_dtypes(include = [int, float]).columns)

In [51]:
pipeline_inicial = make_column_transformer(\
                                           (encoder12, features_qual),
                                           (encoder23, features_quant),
                                           remainder = 'drop'
                       )

pipeline_inicial

ColumnTransformer(transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['city', 'gender', 'relevent_experience',
                                  'enrolled_university', 'education_level',
                                  'major_discipline', 'experience',
                                  'company_size', 'company_type',
                                  'last_new_job']),
                                ('standardscaler', StandardScaler(),
                                 ['city_development_index', 'training_hours'])])

In [52]:
pipeline_com_modelo = make_pipeline(pipeline_inicial, model)

pipeline_com_modelo

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'gender',
                                                   'relevent_experience',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'experience', 'company_size',
                                                   'company_type',
                                                   'last_new_job']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['city_development_index',
                                                   'training_hours'])])),
                ('lgbmclassifier', LGBMClassifier(random_state=42))])

In [53]:
pipeline_com_modelo.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'gender',
                                                   'relevent_experience',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'experience', 'company_size',
                                                   'company_type',
                                                   'last_new_job']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['city_development_index',
                                                   'training_hours'])])),
                ('lgbmclassifier', LGBMClassifier(random_state=42))])

In [55]:
y_pred = pipeline_com_modelo.predict(X_test)

y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [56]:
pd.crosstab(y_test, y_pred, rownames = ['Vida real'], colnames = ['Predito'], margins = True)

Predito,0.0,1.0,All
Vida real,,,
0.0,3733,582,4315
1.0,581,852,1433
All,4314,1434,5748


In [57]:
print(classification_report(y_true = y_test, y_pred = y_pred))

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87      4315
         1.0       0.59      0.59      0.59      1433

    accuracy                           0.80      5748
   macro avg       0.73      0.73      0.73      5748
weighted avg       0.80      0.80      0.80      5748



In [60]:
accuracy_score(y_true = y_test, y_pred = y_pred) * 100

79.7668754349339

In [61]:
recall_score(y_true = y_test, y_pred = y_pred) * 100

59.45568736915562

In [62]:
precision_score(y_true = y_test, y_pred = y_pred) * 100

59.41422594142259

In [63]:
f1_score(y_true = y_test, y_pred = y_pred) * 100

59.43494942448553

## Próximos passos:

1. Tunar o modelo
2. Aplicar os encoders categóricos
3. Cross validation
4. Avaliar as métricas comparando com o novo modelo e a baseline

In [ ]:
# IDEIA DO PIPELINE: Fazer a passo a passo o que o modelo vai fazer
# - Excluir uma variável
# - Aplicar a função que add_feature_null_column
# - Aplicar a função que add_feature_null_qualitative
# - Aplicar a função que add_feature_null_quantitativa
# - Utilizar os encoders nas features qualitativas

Pipeline()

- enrollee_id: excluir
- city: codificar ela de texto -> número inteiro , testar o encoder - tratar os nulos
- city_development_index: manter e tratar os nulos
- gender: testar todos os encoders, tratar os nulos
- relevent_experience: testar todos os encoders, tratar os nulos
- enrolled_university: testar todos os encoders, tratar os nulos
- education_level: testar todos os encoders, tratar os nulos
- major_discipline: testar todos os encoders, tratar os nulos
- experience: testar todos os encoders, tratar os nulos
- company_size: testar todos os encoders, tratar os nulos
- company_type: testar todos os encoders, tratar os nulos
- last_new_job: testar todos os encoders, tratar os nulos
- training_hours: manter e tratar os nulos